In [17]:
import re

def removePunctuation(text):
    text=re.sub("[^0-9a-zA-Z ]", " ", text)
    return text

In [18]:
textRDD = sc.newAPIHadoopFile('../../Data/Moby-Dick.txt',
                              'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
                              'org.apache.hadoop.io.LongWritable',
                              'org.apache.hadoop.io.Text',
                               conf={'textinputformat.record.delimiter': "\r\n\r\n"}) \
            .map(lambda p: p[1])

sentences=textRDD.flatMap(lambda x: x.split(". "))

In [19]:
sentences=sentences.map(removePunctuation).map(lambda x:x.lower())
sentences.take(3)

[u'the project gutenberg ebook of moby dick  or the whale  by herman melville',
 u'this ebook is for the use of anyone anywhere at no cost and with  almost no restrictions whatsoever',
 u' you may copy it  give it away or  re use it under the terms of the project gutenberg license included  with this ebook or online at www gutenberg org']

Note that the input file is generated in Windows. In Windows, the newline character is "\r\n", while in Linux it is "\n". So here we set the delimiter for paragraphs to be "\r\n\r\n" instead of "\n\n".

Refererences:
1. http://stackoverflow.com/questions/17692857/
2. http://stackoverflow.com/questions/7013034/

In [20]:
k=2  # k is part of the environment variables of the function. Spark sends the value of the referenced variables 
     # to the slave nodes.
def ngram(sentence):
    set=[]
    for i in range(len(sentence)-k+1):
        set.append((tuple(sentence[i:i+k]),1))
    return set

ngram(sentences.map(lambda x:x.split()).first())

[((u'the', u'project'), 1),
 ((u'project', u'gutenberg'), 1),
 ((u'gutenberg', u'ebook'), 1),
 ((u'ebook', u'of'), 1),
 ((u'of', u'moby'), 1),
 ((u'moby', u'dick'), 1),
 ((u'dick', u'or'), 1),
 ((u'or', u'the'), 1),
 ((u'the', u'whale'), 1),
 ((u'whale', u'by'), 1),
 ((u'by', u'herman'), 1),
 ((u'herman', u'melville'), 1)]

In [22]:
def printOutput(n,freq_ngramRDD):
    top=freq_ngramRDD.take(5)
    print '\n============ %d most frequent %d-grams'%(5,n)
    print '\nindex\tcount\tngram'
    for i in range(5):
        print '%d.\t%d: \t"%s"'%(i+1,top[i][0],' '.join(top[i][1]))

In [23]:
for k in range(1,6):
    ngrams = sentences.map(lambda x:x.split()).flatMap(ngram)
    freq_ngrams = ngrams.reduceByKey(lambda x,y:x+y).map(lambda x:(x[1],x[0])).sortByKey(False)
    m=5
    printOutput(k,freq_ngrams)


============ 5 most frequent 1-grams

index	count	ngram
1.	14620: 	"the"
2.	6732: 	"of"
3.	6502: 	"and"
4.	4799: 	"a"
5.	4706: 	"to"

============ 5 most frequent 2-grams

index	count	ngram
1.	1906: 	"of the"
2.	1193: 	"in the"
3.	746: 	"to the"
4.	444: 	"from the"
5.	413: 	"the whale"

============ 5 most frequent 3-grams

index	count	ngram
1.	116: 	"the sperm whale"
2.	109: 	"of the whale"
3.	88: 	"the white whale"
4.	64: 	"one of the"
5.	60: 	"of the sea"

============ 5 most frequent 4-grams

index	count	ngram
1.	43: 	"of the sperm whale"
2.	27: 	"the sperm whale s"
3.	20: 	"at the same time"
4.	18: 	"project gutenberg tm electronic"
5.	18: 	"of the whale s"

============ 5 most frequent 5-grams

index	count	ngram
1.	13: 	"the project gutenberg literary archive"
2.	13: 	"project gutenberg literary archive foundation"
3.	12: 	"project gutenberg tm electronic works"
4.	11: 	"of the sperm whale s"
5.	10: 	"and at the same time"
